to be archived

##S3-1: write fasta_genome

In [4]:
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment
from glob import glob
import json
from typing import Generator,Tuple,Optional,Iterable,Callable
import pandas as pd
# from Bio import Align
from pathlib import Path
def iter_match(genome_dict:dict)->Generator[Tuple[dict,dict],None,None]:
    '''
    same itermath as `extract_domains.py`
    put here for convienience
    '''
    for orf in genome_dict['results'][0]['openReadingFrames']:
        for match in orf['protein']['matches']:
            yield (orf,match)

def convert_fasta(gfile:str):
    '''
    gfile: json of interproscan;
    '''
    # o=[]
    o_dict={}
    b=0
        # print('\n\n##'+gfile.split('/')[1].replace(':genome.json',''))
    # genome_name=gfile.split('/')[1].replace(':genome.json','')
    genome_dict=json.load(open(gfile))
    for orf,match in iter_match(genome_dict):
        seq=orf['protein']['sequence']
        if len(match['locations'])==1:
            _=match['locations'][0]
            b,e=_['start'],_['end']
            seq_=seq[b:e]
            name=match['signature']['accession']
            # o.append(f'>{name}\n{seq_}')
            o_dict[name]=seq_
        else:
            for i,_ in enumerate(match['locations']):
                b,e=_['start'],_['end']
                seq_=seq[b:e]
                name=match['signature']['accession']+'#'+str(i)
                # o.append(f'>{name}\n{seq_}')
                o_dict[name]=seq_
    # print('\n'.join(o),file=open('nido-rdrp.fasta','w'))  
    return o_dict

In [11]:
def write_single_genome_fasta(file_limit:str='zika_subset/*:genome.json',
                odir:str='zika_fasta_genome',
                convert:Callable[[str], str]=lambda x: Path(x).name.replace('.json','.fasta'),
                ):
    '''
    `file_limit`: decides json to be extract
    `odir`: dir to save those fasta
    `convert`: callable from glob(file_limit)'s str to filename of output
    '''
    # o={}
    odir=Path(odir)
    odir.mkdir(mode=511,exist_ok=True)
    for i in glob(file_limit):
        o_dict=convert_fasta(i)
        with open(str(odir/convert(i)),'w') as f:
            fastas='\n'.join([f'>{k1}\n{v1}' for k1,v1 in o_dict.items()])
            f.write(fastas)

In [14]:
for i in ['zika','ebola','measles']:
    write_single_genome_fasta(file_limit=f'{i}_subset/*:genome.json',
                odir=f'{i}_fasta_genome')